In [108]:
#install dependences
#!pip install dateutil
#!pip install pandas
#!pip install yfinance
#!pip install pathlib
from datetime import datetime as dt
from datetime import timedelta
import os
import os.path
import pandas as pd
from pathlib import Path
import dateutil.parser as dparser

In [109]:
from datetime import datetime as dt

#data scrubbing and separation

#open master file, separate rows with values in Trading Name from those without
#and drop excluded data to csv as it is done with


df_master = pd.read_csv("OilSpills-Master.csv", index_col=False, header=0)

df_included = df_master[df_master["Trading Name"].notna()]

df_excluded = df_master[df_master["Trading Name"].isna()]

#date correction to production dataframe      
thisYear = int(dt.today().strftime("%Y"))

for index, row in df_included.iterrows():         

    if " " not in row.Dates:

        rawSingle = (dparser.parse(row.Dates))  
        fixedSingle = rawSingle.strftime('%Y-%m-%d')
        fixedDate = fixedSingle + " - " + fixedSingle        

    else:

        rawDoubleSplit = row.Dates.split(" - ")
        rawSplitFirst = (dparser.parse(rawDoubleSplit[0]))
        rawSplitSecond = (dparser.parse(rawDoubleSplit[1]))
        fixedSplitFirst = rawSplitFirst.strftime('%Y-%m-%d')
        fixedSplitSecond = rawSplitSecond.strftime('%Y-%m-%d')
        fixedDate = fixedSplitFirst + " - " + fixedSplitSecond  

    #Fix dates if prior to 1970 for Windows systems 
    splitFixed = fixedDate.split(" - ")

    splitFirstDate = splitFixed[0].split("-")
    splitFirstYear = int(splitFirstDate[0])

    splitSecondDate = splitFixed[1].split("-")
    splitSecondYear = int(splitSecondDate[0])

    if splitFirstYear > thisYear or splitSecondYear > thisYear:

        if splitFirstYear > thisYear:
            splitFirstDate[0] = splitFirstYear - 100
            firstDate = str(splitFirstDate[0]) + "-" + splitFirstDate[1] + "-" + str(splitFirstDate[2])                

        if splitSecondYear > thisYear:
            splitSecondDate[0] = splitSecondYear - 100
            secondDate = str(splitSecondDate[0]) + "-" + splitSecondDate[1] + "-" + str(splitSecondDate[2])

        fixedDate = firstDate + " - " + secondDate


    df_included.at[index, "Dates"] = fixedDate             

In [110]:
#normalize company names in production document to avoid mismatching and other errors
#write sheet of excluded data for reference

import yfinance as yf

#make a dictionary of the ones we've done already to save time
#and not batter the api, which is rude


companyValues = {}

for index, row in df_included.iterrows(): 
            
    ownerName = row["Owner"]
    tradingName = row["Trading Name"]
    
    try:
        companyValues[tradingName]["ticker"]
        ownerNameFixed = companyValues[tradingName]["name"]
        df_included.at[index, "Owner"] = ownerNameFixed
        
    except KeyError:
        
        try:
            dataTicker = yf.Ticker(tradingName)
            dataName = dataTicker.info  
            ownerNameFixed = dataName["shortName"]
            companyValues[tradingName] = {"ticker": tradingName, "name": ownerNameFixed}            
            df_included.at[index, "Owner"] = ownerNameFixed
                
        
        except Exception as exception:
            print(f"There was an error with {tradingName}: {exception}.  Not updated.")
            print("\n")

print("Complete.")


There was an error with DNR: No tables found.  Not updated.


There was an error with EEP: 'regularMarketOpen'.  Not updated.


Complete.


In [111]:
#stock data
from datetime import timedelta
import dateutil.parser as dparser
import os
import os.path
import pandas as pd
import yfinance as yf

if not os.path.exists("stocks"):
    os.makedirs("stocks")

#Get date ranges for each incident (1 day before and 2 years after, if possible) for each incident
#then pull stock data for those data ranges and save to csv.  

for index, row in df_included.iterrows():
    
    tradingName = row["Trading Name"]
    ownerName = row["Owner"]
    dates = row["Dates"]
    dateSplit = dates.split(" - ")     
    
    dateStartDelta = (dparser.parse(dateSplit[0])) - timedelta(days=7)
    dateEndDelta = (dparser.parse(dateSplit[1])) + timedelta(days=737)   
    
    dateStart = dateStartDelta.strftime('%Y-%m-%d')
    dateEnd = dateEndDelta.strftime('%Y-%m-%d') 
            
    #attempt to download the data from yfinance if needed.    
    
    csvTarget = f"stocks/{ownerName} {dateStart} - {dateEnd}.csv"
    
    if not os.path.exists(csvTarget):
    
        try:

            dataTicker = yf.Ticker(tradingName)
            data = dataTicker.history(start=dateStart, end=dateEnd)
            data.stack().reset_index().rename(index=str, columns={"level_1": "Date"})
                     
            data.to_csv(csvTarget, index = True, header=True)  

        except Exception as exception:

            print(f"Exception {exception} for {ownerName} for time period {dateStart} - {dateEnd}")

print("Completed.  Review errors and correct before continuing.")

Completed.  Review errors and correct before continuing.


In [225]:
#Fill in interval adjusted closing values.  If date not listed, get closest date before incident
#and then date closest after for remainder of intervals

for index, row in df_included.iterrows():
    
    ownerName = row["Owner"]
    dates = row["Dates"]
    dateSplit = dates.split(" - ")  
    dateStartDelta = (dparser.parse(dateSplit[0])) - timedelta(days=7)
    dateEndDelta = (dparser.parse(dateSplit[1])) + timedelta(days=737)   
    
    dateStart = dateStartDelta.strftime('%Y-%m-%d')
    dateEnd = dateEndDelta.strftime('%Y-%m-%d') 
    
    dayBeforeDelta = (dparser.parse(dateSplit[0])) - timedelta(days=1)
    dayBefore = dayBeforeDelta.strftime('%Y-%m-%d') 
    
    csvTarget = f"stocks/{ownerName} {dateStart} - {dateEnd}.csv"
    
    datesCSV = pd.read_csv(csvTarget, index_col=False, header=0)
    
    dates = pd.to_datetime(datesCSV["Date"])
    
    #try initial dates.  If these are after the date of the incident, it will be 
    #be apparent in the output and can be corrected.  If no data for date, choose nearest one before  
        
    dayPrior = datesCSV.loc[datesCSV["Date"] == dayBefore]["Close"].tolist()   
    
    if dayPrior == []:
        
        parsedDayBefore= pd.to_datetime(dayBefore)
        fixedDateCalc = min([i for i in dates if i < parsedDayBefore], key=lambda x: abs(x - parsedDayBefore))
        fixedDate = fixedDateCalc.strftime('%Y-%m-%d')

        #print(f"nearest date to {dayBefore} is {fixedDate}  for {csvTarget}.  Check for sanity.") 

        dayPrior = datesCSV.loc[datesCSV["Date"] == fixedDate]["Close"].tolist()
    
    df_included.at[index, "Day Prior"] = dayPrior[0]
    
    #then do the rest as the remainder do not require any supervision after initial sanity check and want nearest date after
        
    dayAfterDelta = (dparser.parse(dateSplit[0])) + timedelta(days=1)
    dayAfter = dayAfterDelta.strftime('%Y-%m-%d') 
    
    oneMonthDelta = (dparser.parse(dateSplit[0])) + timedelta(days=30)
    oneMonth = oneMonthDelta.strftime('%Y-%m-%d')         
        
    sixMonthDelta = (dparser.parse(dateSplit[0])) + timedelta(days=180)
    sixMonth = sixMonthDelta.strftime('%Y-%m-%d')         
    
    oneYearDelta = (dparser.parse(dateSplit[0])) + timedelta(days=365)
    oneYear = oneYearDelta.strftime('%Y-%m-%d') 
    
    twoYearDelta = (dparser.parse(dateSplit[0])) + timedelta(days=730)
    twoYear = twoYearDelta.strftime('%Y-%m-%d') 
    
    tryDict = {dayAfter:"Day After", oneMonth: "1 Month", sixMonth: "6 Months", oneYear: "1 Year", twoYear: "2 Years"}
           
    for key,value in tryDict.items():
       
        targetClose = datesCSV.loc[datesCSV["Date"] == key]["Close"].tolist()
        #print(f"targetClose {targetClose}")
            
        if targetClose == []:
            
            parsedDate = pd.to_datetime(key)
            
            try:
                fixedDateCalc = min([i for i in dates if i > parsedDate], key=lambda x: abs(x - parsedDate))
                fixedDate = fixedDateCalc.strftime('%Y-%m-%d')
                
                targetClose = datesCSV.loc[datesCSV["Date"] == fixedDate]["Close"].tolist()
                print(f"nearest date to {key} is {fixedDate}  for {ownerName}.  Check for sanity.") 
                             
            except:
                
                targetClose = ["NaN"]
                print(f"Date {fixedDate} not in range for {ownerName}.  NaN value placed.")
                
        else:

            targetClose = datesCSV.loc[datesCSV["Date"] == fixedDate]["Close"].tolist()

        #print(f"{tryDict[key]} will equal {targetClose} for {ownerName}.  Verify with random sample.")
        
df_included.to_csv(r"OilSpills-Production.csv", index = False, header=True)  
df_excluded.to_csv(r"OilSpills-Excluded.csv", index =False, header=True)

print("Complete.  Data scrub finished.")

nearest date to 2020-05-30 is 2020-06-01  for MMC NORILSK NICKEL PJSC.  Check for sanity.
nearest date to 2020-06-28 is 2020-06-29  for MMC NORILSK NICKEL PJSC.  Check for sanity.
Date 2020-06-29 not in range for MMC NORILSK NICKEL PJSC.  NaN value placed.
Date 2020-06-29 not in range for MMC NORILSK NICKEL PJSC.  NaN value placed.
Date 2020-06-29 not in range for MMC NORILSK NICKEL PJSC.  NaN value placed.
nearest date to 2019-11-28 is 2019-11-29  for TC Energy Corporation.  Check for sanity.
nearest date to 2020-04-26 is 2020-04-27  for TC Energy Corporation.  Check for sanity.
Date 2020-04-27 not in range for TC Energy Corporation.  NaN value placed.
nearest date to 2019-07-28 is 2019-07-29  for CAP SA.  Check for sanity.
nearest date to 2020-07-26 is 2020-07-27  for CAP SA.  Check for sanity.
Date 2020-07-27 not in range for CAP SA.  NaN value placed.
nearest date to 2018-11-17 is 2018-11-19  for HUSKY ENERGY INC..  Check for sanity.
nearest date to 2018-12-16 is 2018-12-17  for HU